### library imports

In [7]:
import tweepy
# !pip install textblob
from textblob import TextBlob
# !pip install git+https://github.com/JustAnotherArchivist/snscrape.git
import snscrape.modules.twitter as sntwitter
import pandas as pd
import re
import numpy as np

###### tweeter app api key & access token

In [3]:
consumer_key = "1TqW5hse6LaAxCCvzZgORWfhx"
consumer_secret = "lfE5MXH9C8GuhB9J8jeNlQSW9zSQ8zwbQAaZDWqwvz1CWSiEm0"
access_token = "435849217-BZdlW1rVPClgt8m1cXXJdqG6xmCnuJAy6ie5f1MR"
access_token_secret = "ojf8Ak3uvLpOcm6yykfAf11UYpjtood9PXGlDc8DJlqiU"

In [4]:
# OAuthHandler object for authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

# Sets access token and token secret
auth.set_access_token(access_token, access_token_secret)

# Creates API object by passing auth information
api = tweepy.API(auth)

#### Task a: latest trending topic for India. (#tag and no of tweets for the particular #tag)

In [92]:
# twitter india trends by woeid
india_woeid = 2282863
twitter_trends = api.trends_place(id=india_woeid)

trends_list = []

for trend in twitter_trends[0]['trends']:
    trends_list.append([trend['name'], trend['tweet_volume']])

In [102]:
trends_df = pd.DataFrame(trends_list, columns=['#tags', 'Tweets Count'])
trends_df.to_csv('indian_trends.csv', encoding='utf-8', index=True)
trends_df.head(10)

,#tags,Tweets Count
0,Parliament,138126.0
1,#PromiseDay,NaN
2,#BJPGovtDictatingTwitter,97824.0
3,#thursdaymorning,NaN
4,पंडित दीनदयाल उपाध्याय,19419.0
5,Disha Sushant,41068.0
6,एकात्म मानववाद,15522.0
7,BIG EXPOSE,12887.0
8,भारतीय जनसंघ,NaN
9,TAYLOR IS COMING,72007.0


#### Task b:  Extract first 100 tweets for #JoeBiden

In [91]:
query = '#JoeBiden'
count = 100

tweets_list = []

# querry twitter through sntwitter.TwitterSearchScraper and append each tweet to tweets_list array
for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
    if i > count:
        break
    tweets_list.append([tweet.date, tweet.id, tweet.content, tweet.user.username])

In [103]:
# adds tweets_list to data frame
tweets_df = pd.DataFrame(tweets_list, columns=['Date', 'Tweet Id', 'Text', 'Username'])

# saves tweets in csv file
tweets_df.to_csv('100_biden_tweets.csv', encoding='utf-8', index=True)
tweets_df.head(9)

,Date,Tweet Id,Text,Username
0,2021-02-11 05:54:51+00:00,1359742676715802630,I literally haven't been able to find ANY good...,TheRealBennetts
1,2021-02-11 05:54:39+00:00,1359742625885036544,"@POTUS Dear President @POTUS and @VP, most of ...",byambaa89532616
2,2021-02-11 05:54:09+00:00,1359742501901324288,"@POTUS Dear President @POTUS and @VP, most of ...",byambaa89532616
3,2021-02-11 05:52:48+00:00,1359742160824799232,I'm the new Warlord of Mississippi #JoeBiden c...,shayb0y
4,2021-02-11 05:52:06+00:00,1359741984425017345,#JoeBiden had his first call as president with...,AlexGuiden
5,2021-02-11 05:51:34+00:00,1359741848844029957,https://t.co/XcVaW43wdJ #JoeBiden #Myanmar,AmritVichar
6,2021-02-11 05:48:19+00:00,1359741031592325120,No change in policy on Jammu and Kashmir and t...,KhushiiMahajan
7,2021-02-11 05:48:03+00:00,1359740963980251141,Seems to be Like Joe Biden Were given a New #R...,kavita_sharma24
8,2021-02-11 05:47:51+00:00,1359740916672704513,@Oliver29239048 @jactioncenter @AILANational @...,sinak50610617


#### Task c: Sentiment Analysis for #JoeBiden (Is it positive/negative, what is the perception)

In [107]:
# Cleans tweet text by removing links, emails, etc
def clean_tweet_text(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z\t])|(\w+:\/\/\S+)", " ", tweet).split())

tweets_df['Text'] = np.array([clean_tweet_text(tweet) for tweet in tweets_df['Text']])

# Analyse tweet & score 1 for positive, 0 for neutral & -1 for negative sentiment
def sentiment_analyzer(tweet):
    analysis = TextBlob(tweet)
    
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0:
        return 0
    else: 
        return -1
    
tweets_df['sentiment'] = np.array([sentiment_analyzer(tweet) for tweet in tweets_df['Text']])

# saves tweets df with sentiment column to csv file
tweets_df.to_csv('biden_tweets_sentiments.csv', encoding='utf-8', index=True)

tweets_df.head(10)

,Date,Tweet Id,Text,Username,sentiment
0,2021-02-11 05:54:51+00:00,1359742676715802630,I literally haven t been able to find ANY good...,TheRealBennetts,1
1,2021-02-11 05:54:39+00:00,1359742625885036544,Dear President and most of 2020 DV lottery win...,byambaa89532616,1
2,2021-02-11 05:54:09+00:00,1359742501901324288,Dear President and most of 2020 DV lottery win...,byambaa89532616,1
3,2021-02-11 05:52:48+00:00,1359742160824799232,I m the new Warlord of Mississippi JoeBiden ca...,shayb0y,1
4,2021-02-11 05:52:06+00:00,1359741984425017345,JoeBiden had his first call as president with ...,AlexGuiden,1
5,2021-02-11 05:51:34+00:00,1359741848844029957,JoeBiden Myanmar,AmritVichar,0
6,2021-02-11 05:48:19+00:00,1359741031592325120,No change in policy on Jammu and Kashmir and t...,KhushiiMahajan,1
7,2021-02-11 05:48:03+00:00,1359740963980251141,Seems to be Like Joe Biden Were given a New Ro...,kavita_sharma24,1
8,2021-02-11 05:47:51+00:00,1359740916672704513,Brown Dear President and most of 2020 DV lotte...,sinak50610617,1
9,2021-02-11 05:47:31+00:00,1359740830202880000,katyal Dear President and most of 2020 DV lott...,sinak50610617,1


In [108]:
tweets_df['sentiment'].mean()

0.42574257425742573

###### mean is < 1 & > 0 so sentiment is between neutral to positive